_Analysis by Niko McCarty_, February 29, 2020

To the extent possible under law, Niko McCarty has waived all copyright and related or neighboring rights to the code presented in this file under a CC0 1.0 Universal license. You can copy, modify, distribute and perform the work, even for commercial purposes, all without asking permission (though I appreciate attribution). This work is published from: United States.

**To Swim or To Wait?**

Imagine that we have a tiny, micron-scale box. Inside that box is an _E. coli_ cell and a single food particle. The question is this: Is it better for the _E. coli_ cell to swim to the food, or is it better for the _E. coli_ to just sit back, relax, and wait for the food particle to come to it?

To answer this question, we must explore the intersection of diffusion and cellular swimming. This Python analysis is inspired by Problem 12.1 in _Physical Biology of the Cell_, Second Edition, by Rob Phillips _et al_.

Diffusion can be modeled as a random walk, in which a free particle randomly steps left, right, up, down, etc. in a given time. 

An important aspect of diffusion is that each molecule, protein, and so forth has a unique Diffusion Coefficient, D, which depends on the medium in whic

[Fick's Law](https://en.wikipedia.org/wiki/Fick%27s_laws_of_diffusion)

In [1]:
import pandas as pd

#import the Numpy and SciPy modules
import numpy as np
from scipy import stats

#style and interactives
import panel as pn
import colorcet

#plotting
import bokeh
bokeh.io.output_notebook()
from bokeh.io import output_file, show
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import LinearAxis, Range1d
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html

import holoviews as hv
hv.extension('bokeh')

ImportError: cannot import name 'encode_utf8' from 'bokeh.util.string' (C:\Users\corks\Anaconda3\lib\site-packages\bokeh\util\string.py)

In [19]:
# Begin by specifying our variables
v = 30 #microns / second, the velocity of the swimming cell (BioNumbers, "How Fast Do Cells Move?")
D = 600 #microns squared / second, the diffusion coefficient for a sugar molecule in water (BioNumbers 104089)
t = np.linspace(0, 3, 1000)

For just this case, where we have specified variables directly, let us plot diffusion of food vs. swimming of _E. coli_.

In [20]:
#Now that my variables are specified, I want to set up a DataFrame to hold my "time" values.
df2 = pd.DataFrame(data={"time":t})

In [21]:
#Now update our DataFrame to compute vt
df2['distance_swim'] = df2['time'] * v

In [22]:
#Update our DataFrame to compute sqrt(2Dt)
df2['distance_diffusion'] = np.sqrt(2 * D * t)

In [23]:
#Now let us plot diffusion vs. swimming for an E. coli cell!
p1 = bokeh.plotting.figure(
    height=250,
    width=550,
    x_axis_label='time (s)',
    y_axis_label='distance traveled (microns)',
    title = 'diffusion of food particle (grey) vs. swimming E. coli (blue)'
)

p1.line(
    x=t,
    y=df2['distance_swim'],
    line_join='bevel',
    line_width=2,
    color='blue'
)

p1.line(
    x=t,
    y=df2['distance_diffusion'],
    line_join='bevel',
    line_width=2,
    color='grey'
)

bokeh.io.show(p1)

In [43]:
bokeh.io.output_file("output_file_name.html")
bokeh.io.save(p1)

'C:\\Users\\corks\\Dropbox\\git\\interactives\\output_file_name.html'

In [28]:
diffusion_slider = pn.widgets.FloatSlider(name="D", start=10, end=1000, step=10, value=600)
velocity_slider = pn.widgets.FloatSlider(name="v", start=0, end=50, step=1, value=30)

widgets = pn.Column(
    pn.Spacer(height=30), velocity_slider, diffusion_slider, width=300)

In [29]:
@pn.depends(velocity_slider.param.value)
def swim_speed(velocity_slider):
    x = np.linspace(0, 10, 200)
    y = velocity_slider * x
    return hv.Curve(data=(x, y), kdims=["time (seconds)"], vdims=["distance travelled (microns)"])

In [30]:
@pn.depends(diffusion_slider.param.value)
def diffusion_speed(diffusion_slider):
    x = np.linspace(0, 10, 200)
    y = np.sqrt(2 * diffusion_slider * x)
    return hv.Curve(data=(x, y), kdims=["time (seconds)"], vdims=["distance travelled (microns)"])

In [45]:
pn.Row(swim_speed, diffusion_speed, widgets)

Row
    [0] ParamFunction(function)
    [1] ParamFunction(function)
    [2] Column(width=300)
        [0] Spacer(height=30)
        [1] FloatSlider(end=50, name='v', start=0, step=1, value=30)
        [2] FloatSlider(end=1000, name='D', start=10, step=10, value=600)

In [47]:
pn.save('test.html', resources=INLINE)

AttributeError: module 'panel' has no attribute 'save'

Let us now analyze how long it would take 1, 10, 100, 1000 bacteria to consume the glucose molecules in a single cube of sugar.

To do this, we must first compute the rate at which _E. coli_ must take in sugar to proceed with their normal activities. The computation here is quite simple, but we must make a few assumptions.

First, let us assume that a typical _E. coli_ cell divides in roughly 1200 seconds (or 20 minutes), which would be considered quite a rapid growth rate. Furthermore, let us state that a cell weighs approximately $10^{-12}$ grams, or 1 picogram. We arrive at this weight by simply assuming that an _E. coli_ cell is a perfect sphere with dimensions of 1 $\mu m$, thus arriving at a dimension of 1 $ \mu m^{3}$. This multiplied by the density of water (assuming the cell is just a bag of liquid), and we arrive at 1 picogram of mass.

Now, let us further assume that the cell is approximately 1/3 dry mass, and that 1/2 of this dry mass is carbon. Therefore, we would have $\approx 4 * 10^{-13}$ grams of carbon per cell, or roughly $10^{10}$ carbon _atoms_ per cell. Dividing this number by six, and we arrive at the number of sugars that are needed to **build one cell**.

Thus, to meet the "nutritional" requirements of this fast-growing _E. coli_ cell, it would need to take in a staggering **one million glucose molecules every second** to divide in the 20 minute time frame. Wow!

After I calculated this number, I began to think about the real scale of this. What is one million glucose molecules anyway, and how long would an _E. coli_ cell require to, say, consume a single cube of sugar? You know, like the little cubes that you might drop in your coffee.

I Googled these sugar cubes, and it looks like they are usually **2.8 grams** in mass.

Given that glucose has a molecular weight of 180.156 grams per mole, we can calculate that there must be a staggering **$9.3563356 * 10^{21}$** sugar molecules in a single cube.

(Multiply $\frac{2.8 grams }{} \frac{mole}{180.156 grams} \frac{6.02 * 10^{23} molecules}{mole}$)

So, assuming that a single _E. coli_ cell was munching on a sugar cube at its "maximal metabolic activity" (but without the exponential growth), how long would it take that cell to completely eat a single cube of sugar?

It would take $9.3563356 x 10^{15}$ seconds, or a whopping $296649829$ (about 300 million years). 

Now, how long would it take cells that are actually dividing at 20 minutes per division growth to consume that same cube of sugar?

In [32]:
#Set up our parameters
sugar_rate = 60000000 #sugars per minute
doubling_time = 20 #minutes

In [33]:
# The x-values we want
x = np.linspace(1, 2000, 100)

# Tnumber of cells
y = 2 ** (x/doubling_time)

#sugar molecules consumed
y1 = sugar_rate * (y)

In [34]:
p = bokeh.plotting.figure(
    height=250,
    width=550,
    x_axis_label='time elapsed (min)',
    y_axis_label='total cells',
)

p1 = bokeh.plotting.figure(
    height=250,
    width=550,
    x_axis_label='time elapsed (min)',
    y_axis_label='sugar molecules consumed',
)

p.line(
    x=x,
    y=y,
    line_join='bevel',
    line_width=2,
)

p1.line(
    x=x,
    y=y1,
    line_join='bevel',
    line_width=2,
    color='grey',
)

bokeh.io.show(row(p, p1))


A cell population growing exponentially would take about 15 hours to consume all the glucose in that same cube, provided it had ample nitrogen and continued to grow exponentially.

According to a [New Hampshire Public Health report](https://www.dhhs.nh.gov/dphs/nhp/documents/sugar.pdf), the average American today consumes about 152 pounds of sugar every year. With a current population of 327.2 million people, that means that American consumes

**$56574400000$** pounds, or $25661716177328$ grams of sugar per year. That comes out to 

$8.5749868 * 10^{34}$ sugar _molecules_ consumed every year. How fast would an exponentially growing population of cells consume this number of molecules?

Only about 24 hours, I think...